In [1]:
import numpy as np
import pandas as pd
import talib
# import pandas_ta as ta


from mplfinance.original_flavor import candlestick_ohlc
import seaborn as sns
import matplotlib.pyplot as plt
# import matplotlib.dates as mpdates
%matplotlib inline

In [2]:
pair = 'BTC_USDT'
timeframe = '15m'
exchange = 'binance'


df = pd.read_json(f'data/{exchange}/{pair}-{timeframe}.json')
df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
df = df.set_index(['time'])
df['hl2']  = (df['high']+df['low'])/2
df['hlc3'] = (df['high']+df['low']+df['close'])/3
df['ohlc4'] = (df['open']+df['high']+df['low']+df['close'])/4
df = df[-200:]
df.head()

,open,high,low,close,volume,hl2,hlc3,ohlc4
time,,,,,,,,
1661679000000,20030.21,20038.32,20015.00,20029.56,775.73783,20026.660,20027.626667,20028.2725
1661679900000,20029.56,20039.74,19995.57,20020.00,1141.41594,20017.655,20018.436667,20021.2175
1661680800000,20019.14,20033.85,19967.64,20005.26,1249.87806,20000.745,20002.250000,20006.4725
1661681700000,20006.23,20010.14,19961.52,19970.56,1114.59089,19985.830,19980.740000,19987.1125
1661682600000,19970.66,20012.42,19969.00,20003.62,1153.01670,19990.710,19995.013333,19988.9250


In [3]:
def heikenashi(df):
    df['ha_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    df['ha_open'] = (df['open'].shift(1) + df['open'].shift(1)) / 2
    df.iloc[0, df.columns.get_loc("ha_open")] = (df.iloc[0]['open'] + df.iloc[0]['close'])/2
    df['ha_high'] = df[['high', 'low', 'ha_open', 'ha_close']].max(axis=1)
    df['ha_low'] = df[['high', 'low', 'ha_open', 'ha_close']].min(axis=1)
    # df = df.drop(['Open', 'High', 'Low', 'Close'], axis=1)  # remove old columns
    # df = df.rename(columns={"HA_Open": "Open", "HA_High": "High", "HA_Low": "Low", "HA_Close": "Close", "Volume": "Volume"})
    # df = df[['Open', 'High', 'Low', 'Close', 'Volume']]  # reorder columns
    return df

In [4]:
df = heikenashi(df)
df.head()

,open,high,low,close,volume,hl2,hlc3,ohlc4,ha_close,ha_open,ha_high,ha_low
time,,,,,,,,,,,,
1661679000000,20030.21,20038.32,20015.00,20029.56,775.73783,20026.660,20027.626667,20028.2725,20028.2725,20029.885,20038.32,20015.00
1661679900000,20029.56,20039.74,19995.57,20020.00,1141.41594,20017.655,20018.436667,20021.2175,20021.2175,20030.210,20039.74,19995.57
1661680800000,20019.14,20033.85,19967.64,20005.26,1249.87806,20000.745,20002.250000,20006.4725,20006.4725,20029.560,20033.85,19967.64
1661681700000,20006.23,20010.14,19961.52,19970.56,1114.59089,19985.830,19980.740000,19987.1125,19987.1125,20019.140,20019.14,19961.52
1661682600000,19970.66,20012.42,19969.00,20003.62,1153.01670,19990.710,19995.013333,19988.9250,19988.9250,20006.230,20012.42,19969.00


In [5]:

# xvolume = 


In [6]:

        
        

# df['volume_sma'] = talib.SMA(df['volume'], timeperiod=7)
# df['f'] = df['volume_sma']/df['volume']

# df['volume_ema'] = talib.EMA(df['volume'], timeperiod=7)


# dataframe['ema100'] = ta.EMA(dataframe, timeperiod=100)

# # SMA - Simple Moving Average
# dataframe['sma3'] = ta.SMA(dataframe, timeperiod=3)

In [7]:
def tfs_vol_osc(dataframe, src_vol, src_open, src_close):
    df_vol = pd.DataFrame()
    df_vol['org_vol'] = dataframe[src_vol]
    for i in range(dataframe.shape[0]):
        if dataframe[src_close].iloc[i] < dataframe[src_open].iloc[i]:
            dataframe.loc[dataframe.index[i],src_vol] = -dataframe[src_vol].iloc[i] # *= -1
        elif dataframe[src_close].iloc[i] > dataframe[src_open].iloc[i]:
            dataframe.loc[dataframe.index[i],src_vol] = dataframe[src_vol].iloc[i]
        else:
            dataframe.loc[dataframe.index[i], src_vol] = 0
            
    df_vol['sma_vol'] = talib.SMA(dataframe[src_vol], timeperiod=7)
    dataframe['div_vol'] =  df_vol['sma_vol']/7#df_vol['org_vol']
    return dataframe

xclose = 'ha_close'
xopen = 'ha_open'

df = tfs_vol_osc(df, 'volume', xopen, xclose)
df

,open,high,low,close,volume,hl2,hlc3,ohlc4,ha_close,ha_open,ha_high,ha_low,div_vol
time,,,,,,,,,,,,,
1661679000000,20030.21,20038.32,20015.00,20029.56,-775.73783,20026.660,20027.626667,20028.2725,20028.2725,20029.885,20038.32,20015.00,NaN
1661679900000,20029.56,20039.74,19995.57,20020.00,-1141.41594,20017.655,20018.436667,20021.2175,20021.2175,20030.210,20039.74,19995.57,NaN
1661680800000,20019.14,20033.85,19967.64,20005.26,-1249.87806,20000.745,20002.250000,20006.4725,20006.4725,20029.560,20033.85,19967.64,NaN
1661681700000,20006.23,20010.14,19961.52,19970.56,-1114.59089,19985.830,19980.740000,19987.1125,19987.1125,20019.140,20019.14,19961.52,NaN
1661682600000,19970.66,20012.42,19969.00,20003.62,-1153.01670,19990.710,19995.013333,19988.9250,19988.9250,20006.230,20012.42,19969.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661854500000,20456.82,20511.30,20437.43,20500.75,1976.82963,20474.365,20483.160000,20476.5750,20476.5750,20423.090,20511.30,20423.09,127.033378
1661855400000,20500.75,20504.63,20400.23,20405.44,-2899.88653,20452.430,20436.766667,20452.7625,20452.7625,20456.820,20504.63,20400.23,99.984219
1661856300000,20406.29,20439.20,20331.99,20394.09,-3600.75903,20385.595,20388.426667,20392.8925,20392.8925,20500.750,20500.75,20331.99,56.973641


In [8]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'


# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1)

# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        increasing_line_color='green',
        decreasing_line_color='red',
        showlegend=False

    ), row=1, col=1
)

# Colorize the histogram values
colors = np.where(df['div_vol'] < 0, '#a60f28', '#3aa305')

# Plot the histogram
fig.append_trace(
    go.Bar(
        x=df.index,
        y=df['div_vol'],
        name='histogram',
        marker_color=colors,

    ), row=2, col=1
)


# upperx = 30
# upper  = 20
# median = 0
# lower  = -20
# lowerx = -30


# fig.add_hrect(y0=upper, y1=upperx, line_width=0, fillcolor="red", opacity=0.2, row=2, col=1)
# fig.add_hrect(y0=upper, y1=lower, line_width=0, fillcolor="blue", opacity=0.2, row=2, col=1)
# fig.add_hrect(y0=lower, y1=lowerx, line_width=0, fillcolor="green", opacity=0.2, row=2, col=1)


# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)

# Update options and show plot
fig.update_layout(layout)
fig.show()

In [2]:
import numpy as np

In [3]:
d = np.nan


In [5]:
import talib as ta

In [12]:
ta.SMA(np.array([5.0]), 5)

array([nan])